In [1]:

%config Completer.use_jedi = False
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)


# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor




#ploting imports
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv
/kaggle/input/ubiquant-market-prediction/example_test.csv
/kaggle/input/ubiquant-market-prediction/train.csv
/kaggle/input/ubiquant-market-prediction/ubiquant/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/ubiquant-market-prediction/ubiquant/__init__.py


In [2]:
print("XGBoost version:", xgb.__version__)

XGBoost version: 1.5.2


In [3]:
# print('# File sizes')
total_size = 0
start_path = '../input/ubiquant-market-prediction'  # To get size of current directory
for path, dirs, files in os.walk(start_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)
print("Directory size: " + str(round(total_size/ 1000000, 2)) + 'MB')

Directory size: 18546.99MB


In [4]:
%%time
# reduce cols for use to save memory capacity
basic_cols = ['target']
test_cols = ['time_id', 'investment_id']

num_feat = 50
features = [f'f_{i}' for i in range(num_feat)]
cols = basic_cols + features
train_cols = test_cols + features

# load data
train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', usecols=cols)
test_df = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv', usecols=features)
sample_prediction_df = pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")


Data is loaded!
CPU times: user 2min 3s, sys: 10.1 s, total: 2min 13s
Wall time: 4min 46s


In [5]:
print('train shape is {}'.format(train.shape))
print('example_test shape is {}'.format(test_df.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

train shape is (3141410, 51)
example_test shape is (9, 50)
sample_prediction_df shape is (9, 3)


train.head()

In [6]:
# Missing Values Count

missing_values_count = train.isnull().sum()
print (missing_values_count)
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

target    0
f_0       0
f_1       0
f_2       0
f_3       0
f_4       0
f_5       0
f_6       0
f_7       0
f_8       0
f_9       0
f_10      0
f_11      0
f_12      0
f_13      0
f_14      0
f_15      0
f_16      0
f_17      0
f_18      0
f_19      0
f_20      0
f_21      0
f_22      0
f_23      0
f_24      0
f_25      0
f_26      0
f_27      0
f_28      0
f_29      0
f_30      0
f_31      0
f_32      0
f_33      0
f_34      0
f_35      0
f_36      0
f_37      0
f_38      0
f_39      0
f_40      0
f_41      0
f_42      0
f_43      0
f_44      0
f_45      0
f_46      0
f_47      0
f_48      0
f_49      0
dtype: int64
% of missing data =  0.0


In [7]:
X_train = train.loc[:, features]
y_train = train.loc[:, 'target']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)


regressor=xgb.XGBRegressor(
                 tree_learner='serial',
                 n_estimators=1000,
                 num_leaves=64,
                 max_depth=8,
                 learning_rate=0.1,
                 subsample=0.8,
                 feature_fraction=0.6,
                 reg_alpha=0.1,
                 reg_lambda=0.1,
                 random_state=2022,
                 tree_method='gpu_hist' )
regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=1)


In [9]:
cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,
                            task_type='GPU')
cb_model.fit(X_train,y_train,
             eval_set=[(X_test, y_test)],
             use_best_model=True,
             verbose=50)

0:	learn: 0.9193032	test: 0.9201722	best: 0.9201722 (0)	total: 43.7ms	remaining: 21.8s
50:	learn: 0.9134326	test: 0.9161070	best: 0.9161070 (50)	total: 1.69s	remaining: 14.9s
100:	learn: 0.9108417	test: 0.9150357	best: 0.9150357 (100)	total: 3.96s	remaining: 15.6s
150:	learn: 0.9087085	test: 0.9143243	best: 0.9143243 (150)	total: 5.79s	remaining: 13.4s
200:	learn: 0.9067857	test: 0.9137434	best: 0.9137434 (200)	total: 7.46s	remaining: 11.1s
250:	learn: 0.9049108	test: 0.9132739	best: 0.9132739 (250)	total: 9.09s	remaining: 9.02s
300:	learn: 0.9031112	test: 0.9128072	best: 0.9128072 (300)	total: 10.7s	remaining: 7.07s
350:	learn: 0.9013050	test: 0.9124171	best: 0.9124171 (350)	total: 12.3s	remaining: 5.22s
400:	learn: 0.8995698	test: 0.9120965	best: 0.9120965 (400)	total: 13.9s	remaining: 3.44s
450:	learn: 0.8979846	test: 0.9117235	best: 0.9117235 (450)	total: 15.7s	remaining: 1.71s
499:	learn: 0.8964679	test: 0.9114306	best: 0.9114306 (499)	total: 17.3s	remaining: 0us
bestTest = 0.9114

import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    preds = regressor.predict(test_df[features].values)
    sample_prediction_df['target'] = preds
    env.predict(sample_prediction_df)

In [10]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    preds = cb_model.predict(test_df[features].values)
    sample_prediction_df['target'] = preds
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [11]:
sample_prediction_df

,row_id,target
0,1223_0,-0.06152
